In [1]:
#imports
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
import pybaseball
from torch.autograd import Variable
from baseball_scraper import pitching_stats_range

In [2]:
#imports
from pybaseball import team_batting
from pybaseball import team_pitching
from pybaseball import pitching_stats
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
#imports
from baseball_scraper import statcast as bs
from baseball_scraper import playerid_lookup
from baseball_scraper import statcast_pitcher
import mlbgame as mlb
import pandas, numpy
import requests, bs4
import re, os
from datetime import datetime
##pulling data as a data frame
pitching = pitching_stats_range("2021-04-01","2021-08-15")
tbatting = team_batting(2021)
tpitching = team_pitching(2021)

In [5]:
##teams need to be renamed with their initials because
##the two APIs refer to the same team different
init = {"Astros": "HOU",
        "Dodgers": "LAD",
        "Rays": "TBR",
        "White Sox": "CHW",
        "Giants": "SFG",
        "Blue Jays": "TOR",
        "Padres": "SDP",
        "Red Sox": "BOS",
        "Athletics": "OAK",
        "Cubs": "CHC",
        "Mets": "NYM",
        "Yankees": "NYY",
        "Braves": "ATL",
        "Phillies": "PHI",
        "Marlins": "MIA",
        "Brewers": "MIL",
        "Angels": "LAA",
        "Reds": "CIN",
        "Indians": "CLE",
        "Nationals": "WSN",
        "Rangers": "TEX",
        "Cardinals": "STL",
        "Mariners": "SEA",
        "Royals": "KCR",
        "Rockies": "COL",
        "Orioles": "BAL",
        "Tigers": "DET",
        "Twins": "MIN",
        "D-backs": "ARI",
        "Pirates": "PIT"
       }

In [6]:
##initialize the dataframe where data for each team is pulled from
##TO DO: csv
tpitching = tpitching[["teamIDfg", "Team","ERA","WHIP","K/9"]]
tbatting = tbatting[ ["teamIDfg","AVG","OBP","SLG"]]
frame = tpitching.merge(tbatting)
frame.columns = ["teamIDfg", "Team","ERA","WHIP","K9","AVG","OBP","SLG"]
print(frame)

    teamIDfg Team   ERA  WHIP     K9    AVG    OBP    SLG
0         22  LAD  3.13  1.13  10.16  0.247  0.336  0.427
1         30  SFG  3.32  1.12   8.79  0.247  0.327  0.437
2         23  MIL  3.35  1.17  10.16  0.233  0.318  0.400
3         10  OAK  3.64  1.20   8.41  0.238  0.318  0.411
4         21  HOU  3.64  1.21   9.29  0.266  0.338  0.439
5          4  CHW  3.65  1.19  10.41  0.250  0.332  0.419
6         29  SDP  3.68  1.24   9.72  0.247  0.328  0.411
7          9  NYY  3.72  1.18   9.62  0.237  0.326  0.396
8         25  NYM  3.78  1.22   9.59  0.234  0.313  0.380
9         12  TBR  3.79  1.19   9.40  0.236  0.317  0.415
10        20  MIA  3.79  1.25   8.73  0.238  0.305  0.382
11        14  TOR  3.88  1.26   9.35  0.265  0.328  0.458
12        16  ATL  4.09  1.30   9.17  0.242  0.322  0.431
13        28  STL  4.15  1.33   7.88  0.238  0.310  0.394
14         3  BOS  4.29  1.38   9.66  0.260  0.325  0.444
15        11  SEA  4.32  1.28   8.43  0.222  0.299  0.382
16        26  

In [7]:
#make dataframe that will contain all the data from each individual season
cols =  ["Team1ID","p1ERA", "p1WHIP", "p1K9", "t1BA","t1OBP", "t1SLG", "t1ERA", "t1WHIP"
         , "t1K9","Team2ID","p2ERA", "p2WHIP","p2K9", "t2BA", "t2OBP", "t2SLG","t2ERA" , "t2WHIP","t2K9"]
data = pandas.DataFrame(columns = cols)
##data that could be normalized
scaled_data= ["p1ERA","p1WHIP", "p1K9", "t1BA","t1OBP", "t1SLG", "t1ERA", "t1WHIP"
           ,"t1K9","p2ERA", "p2WHIP","p2K9", "t2BA", "t2OBP", "t2SLG","t2ERA" , "t2WHIP","t2K9"]

In [8]:
##helper function that will 
##merge two dictionaries together
def Merge(dict1, dict2):
    return(dict2.update(dict1))

In [9]:
import random
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [10]:
##running through games and adding neccessary data
result = []

for m in range(4,7):
    for d in range(1,31):
        schedules = mlb.games(2021,m,d)
        games = mlb.combine_games(schedules)
        for game in games:
            try:
                ran = random.randint(0,1)
                if ran == 0:
                    t1 = True
                    pitcher1 = game.w_pitcher
                    team1 = game.w_team
                    pitcher2 = game.l_pitcher
                    team2 = game.l_team
                else:
                    t1 = False
                    pitcher2 = game.w_pitcher
                    team2 = game.w_team
                    pitcher1 = game.l_pitcher
                    team1 = game.l_team
            except AttributeError:
                continue
            try:
                team1 = frame.loc[frame['Team'] == init[team1]]
                df = {"Team1ID": team1.teamIDfg.values[0]}
                p1 = pitching.loc[pitching['Name'] == pitcher1]
                p1df = {"p1ERA": p1.ERA.values[0], "p1WHIP": p1.WHIP.values[0], "p1K9": p1.SO9.values[0]}
                Merge(p1df,df)
                t1bdf = {"t1BA": team1.AVG.values[0] ,"t1OBP": team1.OBP.values[0], "t1SLG": team1.SLG.values[0]}
                Merge(t1bdf, df)
                t1pdf = {"t1ERA": team1.ERA.values[0], "t1WHIP": team1.WHIP.values[0], "t1K9": team1.K9.values[0]}
                Merge(t1pdf,df)
                p2 = pitching.loc[pitching['Name'] == pitcher2]
                team2 = frame.loc[frame['Team'] == init[team2]]
                p2df = {"Team2ID": team2.teamIDfg.values[0], "p2ERA": p2.ERA.values[0], "p2WHIP": p2.WHIP.values[0] ,"p2K9": p2.SO9.values[0]}
                Merge(p2df,df)
                t2bdf = {"t2BA": team2.AVG.values[0], "t2OBP": team2.OBP.values[0], "t2SLG": team2.SLG.values[0]}
                Merge(t2bdf,df)
                t2pdf = {"t2ERA": team2.ERA.values[0] , "t2WHIP": team2.WHIP.values[0] ,"t2K9": team2.K9.values[0]}
                Merge(t2pdf,df)
                data = data.append(df,ignore_index=True)
                if t1:
                    result.append([1,0])
                else:
                    result.append([0,1])
            except IndexError:
                continue
print(data)


      Team1ID  p1ERA  p1WHIP  p1K9   t1BA  t1OBP  t1SLG  t1ERA  t1WHIP  t1K9  \
0         9.0   8.79   2.163  13.8  0.237  0.326  0.396   3.72    1.18  9.62   
1         5.0   3.28   1.246  12.9  0.233  0.300  0.402   4.53    1.30  9.33   
2         8.0   7.83   1.649   5.2  0.244  0.318  0.428   4.90    1.33  8.45   
3        27.0   5.76   1.382  12.4  0.235  0.305  0.362   5.08    1.41  8.51   
4        16.0   5.68   1.474   6.6  0.242  0.322  0.431   4.09    1.30  9.17   
...       ...    ...     ...   ...    ...    ...    ...    ...     ...   ...   
1161      7.0   5.35   1.269   8.9  0.241  0.301  0.385   4.94    1.43  8.91   
1162     16.0   3.78   1.286   9.3  0.242  0.322  0.431   4.09    1.30  9.17   
1163     21.0   4.26   0.789   9.2  0.266  0.338  0.439   3.64    1.21  9.29   
1164      8.0   4.53   1.301   9.7  0.244  0.318  0.428   4.90    1.33  8.45   
1165     10.0   3.06   1.033   9.2  0.238  0.318  0.411   3.64    1.20  8.41   

      Team2ID  p2ERA  p2WHIP  p2K9   t2

In [11]:
model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(20,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2,activation = 'softmax')])

In [12]:
data = np.array(data)
result = np.array(result)
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    result,
                                                    test_size=0.33,
                                                    random_state=42)


In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2821 - accuracy: 0.5083
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2453 - accuracy: 0.5570
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2435 - accuracy: 0.5723
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2337 - accuracy: 0.6018
Epoch 5/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2320 - accuracy: 0.6197
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2368 - accuracy: 0.5903
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2294 - accuracy: 0.6120
Epoch 8/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2306 - accuracy: 0.6133
Epoch 9/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2253 - accuracy: 0.6223
Epoch 10/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2213 - accuracy: 0.6364


In [14]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

13/13 - 0s - loss: 0.2180 - accuracy: 0.6701


In [15]:
tdata = pandas.DataFrame(columns = cols)

In [16]:
def fetch_data(team1, pitcher1, team2, pitcher2):
    tdata = pandas.DataFrame(columns = cols)
    team1 = frame.loc[frame['Team'] == init[team1]]
    df = {"Team1ID": team1.teamIDfg.values[0]}
    p1 = pitching.loc[pitching['Name'] == pitcher1]
    p1df = {"p1ERA": p1.ERA.values[0], "p1WHIP": p1.WHIP.values[0], "p1K9": p1.SO9.values[0]}
    Merge(p1df,df)
    t1bdf = {"t1BA": team1.AVG.values[0] ,"t1OBP": team1.OBP.values[0], "t1SLG": team1.SLG.values[0]}
    Merge(t1bdf, df)
    t1pdf = {"t1ERA": team1.ERA.values[0], "t1WHIP": team1.WHIP.values[0], "t1K9": team1.K9.values[0]}
    Merge(t1pdf,df)
    p2 = pitching.loc[pitching['Name'] == pitcher2]
    team2 = frame.loc[frame['Team'] == init[team2]]
    p2df = {"Team2ID": team2.teamIDfg.values[0], "p2ERA": p2.ERA.values[0], "p2WHIP": p2.WHIP.values[0] ,"p2K9": p2.SO9.values[0]}
    Merge(p2df,df)
    t2bdf = {"t2BA": team2.AVG.values[0], "t2OBP": team2.OBP.values[0], "t2SLG": team2.SLG.values[0]}
    Merge(t2bdf,df)
    t2pdf = {"t2ERA": team2.ERA.values[0] , "t2WHIP": team2.WHIP.values[0] ,"t2K9": team2.K9.values[0]}
    Merge(t2pdf,df)
    tdata = tdata.append(df,ignore_index=True)
    tdata = np.array(tdata)
    return tdata

In [28]:
##model that allows team and pitcher input to make a prediction
model.predict(fetch_data("D-backs","Reid Detmers", "Dodgers", "Walker Buehler"), verbose = 2)

1/1 - 0s


array([[0.19598092, 0.80401903]], dtype=float32)

In [17]:
games = mlb.day(2021,8,16)
for a_game in games:
    try:
        ateam = a_game.away_team
        apitcher = a_game.p_pitcher_home
        hteam = a_game.home_team
        hpitcher = a_game.p_pitcher_away
        if apitcher and hpitcher != '.':
            try:
                print(ateam, apitcher, hteam, hpitcher)
                print(model.predict(fetch_data(ateam ,apitcher, hteam, hpitcher), verbose = 2))
            except IndexError:
                print("Pass")
                continue
    except AttributeError:
        continue

Orioles Matt Harvey Rays Collin McHugh
1/1 - 0s
[[0.12998523 0.8700148 ]]
Astros Jake Odorizzi Royals Carlos Hernandez
1/1 - 0s
[[0.4503953 0.5496047]]
Indians Cal Quantrill Twins Griffin Jax
1/1 - 0s
[[0.6854158 0.3145842]]
Athletics Frankie Montas White Sox Dallas Keuchel
1/1 - 0s
[[0.48932981 0.5106702 ]]
Padres Ryan Weathers Rockies Antonio Senzatela
1/1 - 0s
[[0.5176937  0.48230633]]
Mets Rich Hill Giants Kevin Gausman
1/1 - 0s
[[0.25058076 0.7494193 ]]
Pirates Steven Brault Dodgers . 
Pass
